In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
base_path = '../data/raw/' 

df1 = pd.read_csv(base_path + '2021.csv', encoding='latin-1', sep=';')
df2 = pd.read_csv(base_path + '2022.csv', sep=',')
df3 = pd.read_csv(base_path + '2023.csv', encoding='latin-1', sep=';')
df4 = pd.read_csv(base_path + '2024.csv', sep=',')

In [ ]:
colunas = [set(df1.columns), set(df2.columns), set(df3.columns), set(df4.columns)]

differences_found = False

for i in range(len(colunas)):
    for k in range(len(colunas)):
        if i != k:
            diferencas = colunas[i] - colunas[k]
            if diferencas:
                differences_found = True
                print(f"As seguintes colunas estão no dataset {i + 1} e não estão no dataset {k + 1}: {diferencas}")

if not differences_found:
    print("As colunas dos datasets são as mesmas.")

As colunas dos datasets são as mesmas.


In [9]:
df = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)

In [14]:
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,331730.0,2021-01-01,sexta-feira,05:30:00,SP,116,453,REGISTRO,Reação tardia ou ineficiente do condutor,Tombamento,...,0,0,0,1,1,"-24,56168114","-47,86752174",SPRF-SP,DEL05-SP,UOP01-DEL05-SP
1,331804.0,2021-01-01,sexta-feira,08:05:00,AM,174,937,MANAUS,Reação tardia ou ineficiente do condutor,Colisão traseira,...,0,5,1,1,5,"-2,508068","-60,036434",SPRF-AM,DEL01-AM,UOP01-DEL01-AM
2,331815.0,2021-01-01,sexta-feira,10:10:00,CE,222,"128,5",ITAPAJE,Velocidade Incompatível,Tombamento,...,0,0,11,2,12,"-3,707626","-39,623509",SPRF-CE,DEL04-CE,UOP03-DEL04-CE
3,331823.0,2021-01-01,sexta-feira,12:30:00,RJ,493,18,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,Colisão lateral,...,0,1,1,1,3,"-22,6687122","-43,01862696",SPRF-RJ,DEL04-RJ,UOP02-DEL04-RJ
4,331843.0,2021-01-01,sexta-feira,14:40:00,RJ,393,252,BARRA DO PIRAI,Condutor Dormindo,Colisão frontal,...,1,1,1,2,3,"-22,43422023","-43,77561487",SPRF-RJ,DEL05-RJ,UOP04-DEL05-RJ


In [11]:
df.columns

Index(['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional',
       'delegacia', 'uop'],
      dtype='object')

In [15]:
df.shape

(250889, 30)

In [17]:
df.isnull().sum()

id                          0
data_inversa                0
dia_semana                  0
horario                     0
uf                          0
br                          0
km                          0
municipio                   0
causa_acidente              0
tipo_acidente               0
classificacao_acidente      4
fase_dia                    0
sentido_via                 0
condicao_metereologica      0
tipo_pista                  0
tracado_via                 0
uso_solo                    0
pessoas                     0
mortos                      0
feridos_leves               0
feridos_graves              0
ilesos                      0
ignorados                   0
feridos                     0
veiculos                    0
latitude                    0
longitude                   0
regional                   23
delegacia                  85
uop                       219
dtype: int64

Dados nulos a serem tratados: uop, delegacia, regional e classificacao_acidente

In [18]:
df['classificacao_acidente'].value_counts()

classificacao_acidente
Com Vítimas Feridas    189879
Sem Vítimas             42947
Com Vítimas Fatais      18059
Name: count, dtype: int64

### Coluna classificacao_acidente:

Verificando os dados acima, é possível ver que a grande maioria dos acidentes possuem vítimas feridas. Poderíamos substituir os dados nulos da coluna classificacao_acidente pela moda desta coluna, que seria "Com Vítimas Feridas".

Por outro lado, não podemos dizer o que realmente ocorreu neste acidente, então também poderíamos excluir estes dados. Visto que só temos 4 dados nulos nesta coluna.

In [22]:
df['regional'].value_counts()

regional
SPRF-MG    32281
SPRF-SC    29641
SPRF-PR    27239
SPRF-RJ    19499
SPRF-RS    18301
SPRF-SP    17016
SPRF-BA    13279
SPRF-PE    11015
SPRF-ES     8837
SPRF-MT     8524
SPRF-GO     8328
SPRF-DF     7270
SPRF-MS     6383
SPRF-PB     5846
SPRF-CE     5393
SPRF-RO     5324
SPRF-RN     5164
SPRF-PI     4718
SPRF-MA     4114
SPRF-PA     3281
SPRF-AL     2291
SPRF-TO     2276
SPRF-SE     2141
SPRF-AC      936
SPRF-RR      738
SPRF-AP      606
SPRF-AM      423
UniPRF         2
Name: count, dtype: int64

### Coluna 'regional':

Nesta coluna, temos SPRF-XX onde XX é a sigla do estado onde provavelmente o acidente deve ter ocorrido. Neste caso, poderíamos utilizar a coluna 'uf' para fazer este tratamento dos dados.

In [26]:
df['delegacia'].value_counts()

delegacia
DEL01-PR    10079
DEL04-SC     7941
DEL01-SC     7061
DEL01-MG     6712
DEL01-PE     5488
            ...  
DEL02-PI      248
DEL04-PA      241
DEL06-MA      149
SEOP-RR         5
SEOP-AP         1
Name: count, Length: 154, dtype: int64

### Coluna 'delegacia':

Aqui, poderiamos utilizar os XX finais da coluna 'regional' e a moda YY dos numeros DELYY-XX que ja aparecem para cada estado XX. 

In [29]:
df['uop'].value_counts()

uop
UOP01-DEL01-SC    4088
UOP01-DEL01-SP    3858
UOP01-DEL01-PE    3373
UOP03-DEL01-MG    3303
UOP01-DEL02-ES    2767
                  ... 
UOP03-DEL01-AM       7
NPF/DEL08-SP         5
SEOP-RR              5
NPF/DEL01-MT         4
UOP02-DEL02-BA       1
Name: count, Length: 404, dtype: int64

### Coluna 'uop':
Fazer o mesmo que no caso anterior, verificando a moda dos numeros ZZ para os dados UOPZZ-DELYY-XX, onde YY é a moda verificada anteriormente para a delegacia e XX é o estado proveniente da coluna 'uf'.

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250889 entries, 0 to 250888
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      250889 non-null  float64
 1   data_inversa            250889 non-null  object 
 2   dia_semana              250889 non-null  object 
 3   horario                 250889 non-null  object 
 4   uf                      250889 non-null  object 
 5   br                      250889 non-null  int64  
 6   km                      250889 non-null  object 
 7   municipio               250889 non-null  object 
 8   causa_acidente          250889 non-null  object 
 9   tipo_acidente           250889 non-null  object 
 10  classificacao_acidente  250885 non-null  object 
 11  fase_dia                250889 non-null  object 
 12  sentido_via             250889 non-null  object 
 13  condicao_metereologica  250889 non-null  object 
 14  tipo_pista          

## Tratamento do tipo de dados:

Observando as informações acima, pensei nas seguintes mudanças:

- data_inversa: mudar para o formato datetime
- data_inversa (eng de atributos): criar duas colunas chamadas mês e ano, para que possa ser comparado os acidentes na br XXX no mes YY nos 4 anos diferentes, por exemplo.
- horario (eng de atributos): criar periodo do dia: madrugada, manha, tarde e noite, dividindo as 24 horas em 0-6, 6-12, 12-18 e 18-24
- km: trocar para int
- latitude e longitude: trocar para float
- regional, delegacia, uop, classificacao_acidente, uf: trocar para category

In [49]:
df.describe()

,id,br,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
count,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000,250889.000000
mean,487171.674055,210.154291,2.563137,0.083631,0.859133,0.278043,1.028881,0.401165,1.137176,1.985484
std,88135.406738,130.175294,2.116121,0.337769,1.068620,0.617826,1.649065,0.849458,1.193285,1.121350
min,331693.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,404481.000000,101.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,493667.000000,158.000000,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000
75%,563207.000000,324.000000,3.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
max,636019.000000,498.000000,95.000000,23.000000,61.000000,31.000000,61.000000,88.000000,62.000000,131.000000


## Ideias:

- quantidade de acidentes por dia da semana 
- quantidade de acidentes por estado 
- quantidade de acidentes por br
- quantidade de acidentes por dia da semana (por estado e por br)
- quantidade de acidentes por dia da semana (considerando apenas acidentes com óbitos) - (é melhor dirigir na estrada em que dia da semana?)
- quantidade de acidentes por periodo do dia
- quantidade de acidentes no mesmo mes nos diferentes anos - (comparando o mes de abri de 2021, 22, 23 e 24, aumentaram os acidentes em certo estado?)
- quantidade de acidentes por periodo do mes (poderiamos particionar em dia1 - dia10, dia11-dia20, dia21-diafinal) - (existe uma lógica nisso?)
- quantidade de acidentes por dia da semana (considerando acidentes sem feridos - aqueles acidentes mais tranquilos)
